In [ ]:
# !pip install transformers accelerate pillow
# !pip install qwen-vl-utils
# !pip install qwen-vl-utils[decord]==0.0.8
# !pip install deepspeed
# !pip install transformers torch deepspeed accelerate tiktoken
# !pip install transformers==4.49.0
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install python-dotenv openai
!pip install opencv-python
!pip install -U openai-whisper
!pip install sounddevice

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())  # Should print the number of GPUs
print(torch.cuda.get_device_name(0))  # Should print your GPU model


In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-72B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-72B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-72B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-72B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import base64
from pprint import pprint

load_dotenv()

client = OpenAI(
    # If the environment variable is not configured, replace the following line with: api_key="sk-xxx",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
)

image_path_mapo_tofu = "G:\Jupyterstuff\cs437\Smart-Nutrition-Tracking-System\image_classification\PXL_20240406_001802565.RAW-01.COVER.jpg" #easy cultural identification
image_path_goblin_bubble_tea = "G:\Jupyterstuff\cs437\Smart-Nutrition-Tracking-System\image_classification\PXL_20241109_232143865.RAW-01.MP.COVER.jpg" #hard holiday themed drink

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def identify_food(image_path):
    base64_image = encode_image(image_path)

    response = client.chat.completions.create(
        model="qwen-vl-plus",
        messages=[{"role": "user","content": [
                {"type": "text","text": """You are a food classification system that reviews images of food and provides a standard name for them which can be cross referenced to provide nutrition information.  Given an image, you will provide the label for the food.  As a classifier, you are to provide your response only as a string in a JSON response.  For example, when given an apple, you should say `{"label": "apple"}`, and nothing else.
                what is this?"""},
                {"type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]}]
        )
    return response
    
test_images = [image_path_mapo_tofu,image_path_goblin_bubble_tea]
for image_path in test_images:
    response = identify_food(image_path)
    pprint(response.choices[0].message.content)

In [ ]:
print(response.usage)

In [ ]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="llama-3.2-11b-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/f/f2/LPU-v1-die.jpg"
                    }
                }
            ]
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=False,
    stop=None,
)

print(completion.choices[0].message)

In [ ]:
import cv2
import time
from picamera2 import Picamera2

def show_camera_feed():
    # Initialize the camera
    picam2 = Picamera2()
    picam2.configure(picam2.create_preview_configuration(main={"size": (800, 600)}))
    picam2.start()
    
    # Allow camera to warm up
    time.sleep(2)
    
    print("Camera opened successfully. Press 'q' to quit.")
    
    # Continuous loop to display camera feed
    while True:
        # Capture frame
        frame = picam2.capture_array()
        
        # Display the resulting frame
        cv2.imshow('Camera Feed', frame)
        
        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Close all windows
    cv2.destroyAllWindows()

if __name__ == "__main__":
    show_camera_feed()

In [ ]:
from picamera2 import Picamera2
from time import sleep

# Initialize camera
picam2 = Picamera2()
config = picam2.create_still_configuration()
picam2.configure(config)

# Start camera
picam2.start()
sleep(2)  # Give time to adjust

# Capture image
picam2.capture_file("test.jpg")

# Stop camera
picam2.close()

In [ ]:
from picamera2 import Picamera2
picam2 = Picamera2()
config = picam2.create_preview_configuration()
picam2.configure(config)
picam2.start()
# Wait a bit for the camera to start
import time
time.sleep(2)
# Capture an image
picam2.capture_file("test.jpg")
picam2.stop()
print("Image captured successfully!")

In [ ]:
#show raspberry pi video feed in jupyter notebook
from picamera2 import Picamera2
from IPython.display import display, Image, clear_output
import time
import io

# Initialize camera
picam2 = Picamera2()
config = picam2.create_preview_configuration()
picam2.configure(config)
picam2.start()

# Give camera time to initialize
time.sleep(2)

try:
    # Display live feed
    while True:  # Display 30 frames
        # Capture to in-memory file
        stream = io.BytesIO()
        picam2.capture_file(stream, format='jpeg')
        stream.seek(0)
        
        # Display the image
        clear_output(wait=True)
        display(Image(data=stream.getvalue()))
        time.sleep(0.1)  # Adjust for desired frame rate
finally:
    picam2.stop()